In [1]:
import numpy as np

In [2]:
def print_f(state):
    for i in range(5):
        for j in range(5):
            y = ''.join([str(int(k)) for k in state[i,j,:]])
            print(y)
        print()

In [3]:
key = "606160E8000000008080820269656CE9E1EBEBEE69656CE9818A8B0669656CE9010A090400000000818A8B0669656CE9616B69EC00000000606160E800000000"

In [4]:
hexa = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
b = 1600
l = 512
c = 1024
r = 576
rounds = 24
def sha3(inp):
    state = np.zeros([5,5,64])
    tempstate = np.zeros([5,5,64])

    message = np.zeros(r)

    k = 0
    for i in range(len(inp)):
        t = [int(j) for j in list('{0:08b}'.format(ord(inp[i])))]
        for j in t:
            message[k] = j
            k+=1

    for k in range(r):
        state[k//(64*5)][(k//64) % 5][k%64] = message[k]

    current_round = 0
    column_parity = np.zeros([5,64])
    print_f(state)
    while current_round < rounds:
        #print("round:",current_round)
        #print_f(state)
        #theta operation
        for i in range(5):
            for k in range(64):
                column_parity[i][k] = int(state[i][0][k])^int(state[i][1][k])^int(state[i][2][k])^int(state[i][3][k])^int(state[i][4][k])

        for i in range(5):
            for j in range(5):
                for k in range(64):
                    state[i][j][k] = int(state[i][j][k])^(int(column_parity[(i+4)%5][k])^int(column_parity[(i+1)%5][k]))
                    tempstate[i][j][k] = state[i][j][k]
        #print("theta:")
        #print_f(state)
        #pi operation
        for i in range(5):
            for j in range(5):
                for k in range(64):
                    state[j][((2 * i) + (3 * j)) % 5][k] = tempstate[i][j][k]
        #print("pi:")
        #print_f(state)
        #chi operation
        tempstate = np.copy(state)

        for i in range(5):
            for j in range(5):
                for k in range(64):
                    state[i][j][k] = int(tempstate[i][j][k])^(~int(tempstate[i][(j+1)%5][k]) & int(tempstate[i][(j+2)%5][k]))
        #print("chi:")
        #print_f(state)
        current_round+=1

    #print(state)
    k = 0
    out = ''
    while k < 1600:
        index = 0
        for j in range(3,-1,-1):
            index = index*2 + state[k//(64*5)][(k//64)%5][k%64 + j]
        out+=hexa[int(index)]
        k += 4
        if k%64==0:
            out+="\n"
    return out

In [5]:
chi_map={}
for i in range(32):
    col = [int(j) for j in '{0:05b}'.format(i)]
    col_new = []
    for j in range(5):
        col_new.append(int(col[j])^(~int(col[(j+1)%5]) & int(col[(j+2)%5])))
    chi_map[(''.join([str(int(k)) for k in col_new]))] = [int(j) for j in '{0:05b}'.format(i)]
print("chi map:",chi_map)

#pi_inversion
def pi_inv(tempstate):
    state = np.zeros([5,5,64])
    for i in range(5):
        for j in range(5):
            for k in range(64):
                state[i][j][k] = tempstate[j][((2*i)+(3*j))%5][k]
    #print("pi inv:")
    #print_f(state)
    return state

#chi_inversion
def chi_inv(tempstate):
    state = np.zeros([5,5,64])
    for i in range(5):
        for j in range(64):
            curr = ''.join([str(int(k)) for k in tempstate[i,:,j].tolist()])
            #print(curr)
            curr = chi_map[curr]
            for k in range(5):
                state[i][k][j]=curr[k]
    #print("chi inv:")
    #print_f(state)
    return state

#theta_inversion
def theta_inv(state):
    column_parity = np.zeros([5,64])
    for i in range(5):
        for i in range(5):
            for k in range(64):
                column_parity[i][k] = int(state[i][0][k])^int(state[i][1][k])^int(state[i][2][k])^int(state[i][3][k])^int(state[i][4][k])
        for i in range(5):
            for j in range(5):
                for k in range(64):
                    state[i][j][k] = int(state[i][j][k])^(int(column_parity[(i+4)%5][k])^int(column_parity[(i+1)%5][k]))                    
    #print("theta inv:")
    #print_f(state)
    return state

chi map: {'00000': [0, 0, 0, 0, 0], '00101': [0, 0, 0, 0, 1], '01010': [0, 0, 0, 1, 0], '01011': [0, 0, 0, 1, 1], '10100': [0, 0, 1, 0, 0], '10001': [0, 0, 1, 0, 1], '10110': [0, 0, 1, 1, 0], '10111': [0, 0, 1, 1, 1], '01001': [0, 1, 0, 0, 0], '01100': [0, 1, 0, 0, 1], '00011': [0, 1, 0, 1, 0], '00010': [0, 1, 0, 1, 1], '01101': [0, 1, 1, 0, 0], '01000': [0, 1, 1, 0, 1], '01111': [0, 1, 1, 1, 0], '01110': [0, 1, 1, 1, 1], '10010': [1, 0, 0, 0, 0], '10101': [1, 0, 0, 0, 1], '11000': [1, 0, 0, 1, 0], '11011': [1, 0, 0, 1, 1], '00110': [1, 0, 1, 0, 0], '00001': [1, 0, 1, 0, 1], '00100': [1, 0, 1, 1, 0], '00111': [1, 0, 1, 1, 1], '11010': [1, 1, 0, 0, 0], '11101': [1, 1, 0, 0, 1], '10000': [1, 1, 0, 1, 0], '10011': [1, 1, 0, 1, 1], '11110': [1, 1, 1, 0, 0], '11001': [1, 1, 1, 0, 1], '11100': [1, 1, 1, 1, 0], '11111': [1, 1, 1, 1, 1]}


In [6]:
#debug
l3 = sha3('lovestarfish').split("\n")[:-1]
l2 = l3[:10]
#problem
l1 = [key[i:i+16] for i in range(0,128,16)]
l1.append('0000000000000000')
l1.append('E0E1E2EA69656CE9')
#use in l3
l4 = []
for i in l3:
    l4.append(l1[l2.index(i)])

0110110001101111011101100110010101110011011101000110000101110010
0110011001101001011100110110100000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000

0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000

0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000

0000000000000000000000

In [7]:
#debug
state = np.zeros([5,5,64])
print(l3)
for j in range(len(l3)):
    temp = [('{0:04b}'.format(hexa.index(i)))[::-1] for i in l3[j]]
    temp = ''.join(temp)
    for i in range(len(temp)):
        state[j//5][j%5][i]=int(temp[i])
#print_f(state)
current_round = 0
while current_round<rounds:
    #print("round:",current_round)
    state = theta_inv(pi_inv(chi_inv(state)))
    current_round+=1

state = state.tolist()

y = ''
for i in state[0][0:2]:
    x = ''.join([str(int(j)) for j in i])
    print(x)
    for j in range(0,64,8):
        y += chr(int(x[j:j+8],2))
y[:12]

['6269E46000000000', '0106020AECE268E4', '676FEE6BECE268E4', '05060A0BECE268E4', '0400080100000000', '05060A0BECE268E4', '6669EC6100000000', '6269E46000000000', '0000000000000000', '636FE66AECE268E4', '0000000000000000', '6269E46000000000', '636FE66AECE268E4', '0400080100000000', '0000000000000000', '05060A0BECE268E4', '05060A0BECE268E4', '0400080100000000', '0000000000000000', '0106020AECE268E4', '0106020AECE268E4', '676FEE6BECE268E4', '676FEE6BECE268E4', '676FEE6BECE268E4', '0400080100000000']
0110110001101111011101100110010101110011011101000110000101110010
0110011001101001011100110110100000000000000000000000000000000000


'lovestarfish'

In [8]:
#final
state = np.zeros([5,5,64])
print(l4)
for j in range(len(l4)):
    temp = [('{0:04b}'.format(hexa.index(i)))[::-1] for i in l4[j]]
    temp = ''.join(temp)
    for i in range(len(temp)):
        state[j//5][j%5][i]=int(temp[i])
print_f(state)
current_round = 0
while current_round<rounds:
    #print("round:",current_round)
    state = theta_inv(pi_inv(chi_inv(state)))
    current_round+=1

state = state.tolist()

y = ''
for i in state[0][0:2]:
    x = ''.join([str(int(j)) for j in i])
    print(x)
    for j in range(0,64,8):
        y += chr(int(x[j:j+8],2))
y[:12]

['606160E800000000', '8080820269656CE9', 'E1EBEBEE69656CE9', '818A8B0669656CE9', '010A090400000000', '818A8B0669656CE9', '616B69EC00000000', '606160E800000000', '0000000000000000', 'E0E1E2EA69656CE9', '0000000000000000', '606160E800000000', 'E0E1E2EA69656CE9', '010A090400000000', '0000000000000000', '818A8B0669656CE9', '818A8B0669656CE9', '010A090400000000', '0000000000000000', '8080820269656CE9', '8080820269656CE9', 'E1EBEBEE69656CE9', 'E1EBEBEE69656CE9', 'E1EBEBEE69656CE9', '010A090400000000']
0110000001101000011000000111000100000000000000000000000000000000
0001000000010000000101000000010001101001011010100110001101111001
0111100001111101011111010111011101101001011010100110001101111001
0001100000010101000111010000011001101001011010100110001101111001
0000100000000101000010010000001000000000000000000000000000000000

0001100000010101000111010000011001101001011010100110001101111001
0110100001101101011010010111001100000000000000000000000000000000
0110000001101000011000000111000100000000000

'pxtuijcyhmis'